In [1]:
import pandas as pd
import numpy as np

def escrever(dfa, nome):
    dfa.to_excel('%s.xlsx' %nome, index=False)

# Ler e Mudar nomes

Juntar tudo na mesma base de dados.
Já temos os produtos para o piloto, fazer a base para machine learning (10 dias antes) e as lojas.

In [2]:
#Stocks, Linear, 
dfDelta=pd.read_excel("Stocks_fev.xlsx", sheet_name="Sheet1")

#Info Ninja
dfNinjas=pd.read_excel("Resultado Sonae_fevereiro 23.xlsx")

#Sellouts, vendedor e reposição
dfSonae1=pd.read_excel("Sellout_fev.xlsx")
dfSonae2=pd.read_excel("Reposição 2022.xlsx")
dfSonae1['Reposição'] = [1 if val in dfSonae2['Descrição Loja'].values else 0 for val in dfSonae1['Descrição Loja']]


dfNinjas = dfNinjas.rename(columns={' Data de Resposta': 'DATA', "Código da loja":"STORE"})
dfSonae = dfSonae.rename(columns={'Data': 'DATA', "LojaEntreposto":"STORE", "DscArtigo":"DESC_ARTIGO"})

# VER

In [18]:
#dfDelta.head()
#dfNinjas.head()
#dfSonae.head()

,DATA,STORE,Loja,Unnamed: 3,DscArtigo,Ean,Quant,Valor,ValorOnline
0,24/02/2023,L0006,CNT COIMBRASHOPPING,NaN,CAFÉ DELTA Q AQTIVUS 10CAP,5601082026978,26,81.64,3.14
1,24/02/2023,L3864,CBD CALDAS TAIPAS,NaN,CAFÉ BELLISSIMO INTENSO 200GR,5601082049946,1,1.89,0.00
2,24/02/2023,L0927,CNT BARREIRO,NaN,CAFÉ DELTA Q ROAST COL DARK 10CAP,5601082047546,1,3.44,0.00
3,24/02/2023,L0459,CNT BRAGA,NaN,CAFÉ MOÍDO SACO CHÁVENA DELTA 250G,5601082022765,32,76.80,0.00
4,24/02/2023,L0446,CNT LOULÉ,NaN,INFUSÃO TETLEY IMUNE VIT LIMAO 20SAQ,5014328001033,3,8.52,0.00


# Mudar para datetime os que forem precisos e remover L's e 0's

- Datetime

In [3]:
dfDelta['DATA'] = pd.to_datetime(dfDelta['DATA'])
dfSonae['DATA'] = pd.to_datetime(dfSonae['DATA'])

- Limpar

In [4]:
dfSonae['STORE'] = dfSonae['STORE'].str.replace('L', '')
dfSonae['STORE'] = dfSonae['STORE'].astype('int')

# Pedir dados fundamentais

In [24]:
x= int(input("Número de colunas que não são produto: "))
y= int(input("A partir de qual coluna aparece produto: "))
n=len(dfNinjas.columns)-x

nomesProd=dfNinjas.columns[y-1:y-1+n]
nomesResto=dfNinjas.columns.difference(nomesProd)

Número de colunas que não são produto: 5
A partir de qual coluna aparece produto: 5


# Código Ninjas (pôr long)

In [6]:
dfNinjasLong = dfNinjas.melt(id_vars=nomesResto, value_vars=nomesProd, var_name='DESC_ARTIGO', value_name='NinjaInfo')

# Código para Delta (Stocks)

In [7]:
dfMeio= pd.merge(dfNinjasLong, dfDelta, how="left", on = ["DATA","STORE","DESC_ARTIGO"]) 
dfMeio=dfMeio.drop(columns=["ar_stor","STORE_NAME","Enc PBL + DIRECTO"])

# Código para Sonae (Sellouts)

In [10]:
dfFinal= pd.merge(dfMeio, dfSonae, how="left", on = ["DATA","STORE","DESC_ARTIGO"])
dfFinal=dfFinal.drop(columns=["Unnamed: 3","Ean","Loja_y"])
dfFinal=dfFinal.rename(columns={'Loja_x': 'Loja'})

# Já temos DataFrame com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição

## Fazer:

- Rotura
- Sinal -> Avaliação da informação ninja vs stock
- Ciclos
- Adequação de Stock

> Rotura

In [ ]:
dfFinal["Rotura"] = np.where(dfFinal["SOH"] > 0, 0, 1)

>Sinal

In [22]:
dfFinal["Sinal"]= pd.Series(dtype='int')

dfFinal['Sinal'] = np.where(dfFinal['NinjaInfo'].astype(bool) & dfFinal['SOH'].astype(bool), 1,
                         np.where(~dfFinal['NinjaInfo'].astype(bool) & dfFinal['SOH'].astype(bool), 2,
                                  np.where(dfFinal['NinjaInfo'].astype(bool) & ~dfFinal['SOH'].astype(bool), 3, 4)))

# 1: Há Stock e Sinal Positivo Ninja
# 2: Há Stock e Sinal Negativo Ninja
# 3: Não há Stock e Sinal Positivo Ninja
# 4: Não há Stock e Sinal Negativo Ninja

> Ciclos

In [ ]:
dfFinal["Ciclos"]=dfFinal["SOH"]/dfFinal["PRES_STOCK"]

> Adequação

In [ ]:
dfFinal["Adequação"]= np.where(dfFinal["Ciclos"] > 1.1, "Stock Suficiente", 
                               np.where(dfFinal["Ciclos"] <= 1.1 & dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["SOH"]>=dfFinal["PRES_STOCK"], "Stock Insuf c Forn Adequado", 
                                np.where(dfFinal["Ciclos"] <= 1.1 & & dfFinal["INTRANSIT"]+dfFinal["EXPECTED"]+dfFinal["SOH"]<dfFinal["PRES_STOCK"], "Stock Insuf c Forn Desadequado")))


In [ ]:
for i in nomesProd:
    adequa = "Adequação de %s" % i
    ciclos = "Ciclos %s" % i
    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "SOH %s" % i
    linear = "PRES_STOCK %s" % i
    dfFinal[adequa] = np.where(dfFinal[ciclos] > 1.1, "Stock Suficiente", 
                               np.where(dfFinal[ciclos] <= 1.1 & dfFinal[transito]+dfFinal[esperado]+dfFinal[stock]>=dfFinal[linear], "Stock Insuf c Forn Adequado", 
                                np.where(dfFinal[ciclos] <= 1.1 & & dfFinal[transito]+dfFinal[esperado]+dfFinal[stock]<dfFinal[linear], "Stock Insuf c Forn Desadequado")))

# Acabamos com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição
- Rotura
- Sinal 
- Ciclos
- Adequação de Stock

# Reomear e organizar

In [25]:
dfFinal.head() #O que é o Artigo?

,DATA,Hora da visita,Loja,Quantidade de produtos Delta presentes,STORE,DESC_ARTIGO,NinjaInfo,ARTIGO,SOH,INTRANSIT,EXPECTED,PRES_STOCK,VND (D-1),EAN,Quant,Valor,ValorOnline,Estado
0,2023-02-25,09:59:00,Continente Amadora,5,2,PACK CAFÉ DELTA Q GRAND MYTHIQ 72+8 CAP,1,7632237.0,133.0,0.0,60.0,6.0,7.0,5.601082e+12,11.0,227.59,0.00,1
1,2023-02-24,13:17:00,Continente Antas,5,14,PACK CAFÉ DELTA Q GRAND MYTHIQ 72+8 CAP,1,7632237.0,100.0,0.0,0.0,6.0,1.0,5.601082e+12,1.0,20.69,0.00,1
2,2023-02-26,19:29:00,Continente Aveiro,3,458,PACK CAFÉ DELTA Q GRAND MYTHIQ 72+8 CAP,1,7632237.0,40.0,0.0,6.0,6.0,4.0,5.601082e+12,4.0,82.76,20.69,1
3,2023-02-25,19:58:00,Continente Barreiro,2,927,PACK CAFÉ DELTA Q GRAND MYTHIQ 72+8 CAP,0,7632237.0,27.0,0.0,0.0,6.0,2.0,5.601082e+12,1.0,20.69,0.00,2
4,2023-02-25,18:59:00,Continente Beja,5,207,PACK CAFÉ DELTA Q GRAND MYTHIQ 72+8 CAP,1,7632237.0,235.0,0.0,12.0,6.0,10.0,5.601082e+12,13.0,268.97,0.00,1


In [30]:
dfFinal=dfFinal.rename(columns={"VND (D-1)": "1_Dia_Antes", "Quant": "Sellout_actual"})
dfFinalCorr=dfFinal.iloc[:, [0,1,2,4,3,5,13,7,6,8,9,10,11,14,12,15,16,17]]

# Escrever

In [44]:
#dfFinalCorr.to_excel('ficheiro_NSS.xlsx', index=False)
#dfFinalCorr.to_csv('ficheiro_NSS.csv', index=False)